# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [127]:
pip install datasets peft

Note: you may need to restart the kernel to use updated packages.


In [128]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, \
    TrainingArguments, Trainer, default_data_collator
from sklearn.metrics import accuracy_score, f1_score

In [129]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Set padding token ID in the model config
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [130]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model device: ", next(model.parameters()).device)

Model device:  cuda:0


In [131]:
dataset = load_dataset('ag_news')
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [132]:
local_pc = True

In [133]:
if local_pc:
    train_subset = dataset['train'].select(range(10))
    test_subset = dataset['test'].select(range(1))
    print(f"Train dataset size: {len(train_subset)}")
    print(f"Test dataset size: {len(test_subset)}")
else:
    train_subset = dataset['train']
    test_subset = dataset['test']

Train dataset size: 10
Test dataset size: 1


In [134]:
def preprocess_data(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

encoded_train_subset = train_subset.map(preprocess_data, batched=True)
encoded_test_subset = test_subset.map(preprocess_data, batched=True)

encoded_train_subset = encoded_train_subset.rename_column("label", "labels")
encoded_test_subset = encoded_test_subset.rename_column("label", "labels")

In [135]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': accuracy, 'f1': f1}

In [136]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_subset,
    eval_dataset=encoded_test_subset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

c:\Users\hp\anaconda3\envs\torch\Lib\site-packages\transformers\training_args.py:1522: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [137]:
# Evaluating the original model
results = trainer.evaluate(encoded_test_subset)
print(f"Foundation Model Results: {results}")

  0%|          | 0/1 [00:00<?, ?it/s]

Foundation Model Results: {'eval_loss': 9.180270195007324, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.255, 'eval_samples_per_second': 3.922, 'eval_steps_per_second': 3.922}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [138]:
from peft import get_peft_model, LoraConfig, TaskType, AutoPeftModelForSequenceClassification, PeftModel

In [139]:
print(list(TaskType))

[<TaskType.SEQ_CLS: 'SEQ_CLS'>, <TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>, <TaskType.CAUSAL_LM: 'CAUSAL_LM'>, <TaskType.TOKEN_CLS: 'TOKEN_CLS'>, <TaskType.QUESTION_ANS: 'QUESTION_ANS'>, <TaskType.FEATURE_EXTRACTION: 'FEATURE_EXTRACTION'>]


In [140]:
# Create a PEFT config
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=2,  # rank
    lora_alpha=16,
    lora_dropout=0.1,
    bias='none',
)

In [141]:
# Create a PEFT model
peft_model = get_peft_model(model, peft_config)

c:\Users\hp\anaconda3\envs\torch\Lib\site-packages\peft\tuners\lora\layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [142]:
# Train the PEFT model
print("Fine-tuning the PEFT model...")
trainer.model = peft_model
trainer.train()

Fine-tuning the PEFT model...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 9.046016693115234, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.0621, 'eval_samples_per_second': 16.092, 'eval_steps_per_second': 16.092, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 8.818970680236816, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.056, 'eval_samples_per_second': 17.857, 'eval_steps_per_second': 17.857, 'epoch': 4.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 8.798376083374023, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.054, 'eval_samples_per_second': 18.521, 'eval_steps_per_second': 18.521, 'epoch': 5.0}
{'train_runtime': 33.5226, 'train_samples_per_second': 1.492, 'train_steps_per_second': 0.298, 'train_loss': 8.014313507080079, 'epoch': 5.0}


TrainOutput(global_step=10, training_loss=8.014313507080079, metrics={'train_runtime': 33.5226, 'train_samples_per_second': 1.492, 'train_steps_per_second': 0.298, 'total_flos': 13076869939200.0, 'train_loss': 8.014313507080079, 'epoch': 5.0})

In [143]:
# Save the trained PEFT model
peft_model.save_pretrained('./peft_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [144]:
# Load the trained PEFT model
trained_peft_model = AutoPeftModelForSequenceClassification.from_pretrained('./peft_model', num_labels=4, pad_token_id=tokenizer.eos_token_id)
#TODO:  use AutoPeftModelForSequenceClassification instead of PeftModel
# trained_peft_model = PeftModel.from_pretrained(trained_peft_model, './peft_model')
trained_peft_model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=2, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=2, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterD

In [145]:
print("Evaluating the fine-tuned PEFT model...")
trainer.model = trained_peft_model
peft_results = trainer.evaluate()
print(peft_results)

Evaluating the fine-tuned PEFT model...


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 8.798376083374023, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.1846, 'eval_samples_per_second': 5.417, 'eval_steps_per_second': 5.417, 'epoch': 5.0}
